Cargas librerías

In [58]:
! pip install mlxtend


[notice] A new release of pip available: 22.1.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [59]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
from mlxtend.preprocessing import TransactionEncoder 
import re

In [60]:
#### conectar_base_de_Datos

conn=sql.connect('db_movies')
cur=conn.cursor()

In [61]:
#### ver tablas disponibles en base de datos ###

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',)]

Sistema de recomendación basado en contenido un solo producto - Manual

In [62]:
#Se crea tabla master
movies=pd.read_sql('select * from movies_final', conn )

In [63]:
#Se realiza el .ifno para observar detalles de la tabla
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2441 entries, 0 to 2440
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  2441 non-null   int64 
 1   title    2441 non-null   object
 2   genres   2441 non-null   object
dtypes: int64(1), object(2)
memory usage: 57.3+ KB


In [64]:
# se convierte la columna genres en variables dummies
genres=movies['genres'].str.split('|')

te = TransactionEncoder()

genres = te.fit_transform(genres)

genres = pd.DataFrame(genres, columns = te.columns_)

movies = pd.concat([movies, genres], axis=1).drop(columns=["genres"])

movies.iloc[:,2:]=movies.iloc[:,2:].astype(int)

In [65]:

#Separar año del titulo
regex = r"\([0-9]+\)"
movies["year"]=movies.title.apply(lambda x:re.search(regex, x).group(0).replace("(","").replace(")",""))
movies["title"]=movies.title.apply(lambda x:re.sub(regex, "",x))

In [66]:
#Escalonar year
sc=MinMaxScaler()
movies[["year"]]=sc.fit_transform(movies[['year']])

In [67]:
genres

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,False,True,True,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
1,False,True,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2436,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2437,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
2438,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2439,True,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False


Sistema de recomendacion

In [68]:
def recomendacion(movie = list(movies['title'])):
     
    ind_libro=movies[movies['title']==movie].index.values.astype(int)[0]   
    similar_movies = genres.corrwith(genres.iloc[ind_libro,:],axis=1)
    similar_movies = similar_movies.sort_values(ascending=False)
    top_similar_movies=similar_movies.to_frame(name="correlación").iloc[0:11,]
    top_similar_movies['title']=movies["title"]
    
    return top_similar_movies


print(interact(recomendacion))

interactive(children=(Dropdown(description='movie', options=('Toy Story ', 'Jumanji ', 'Grumpier Old Men ', 'F…

<function recomendacion at 0x00000184F6FFC4C0>


Sistema de recomendación basado en contenido KNN un solo producto visto

In [72]:
#Entrenar modelo
model = neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
model.fit(genres)
dist, idlist = model.kneighbors(genres)

distancias=pd.DataFrame(dist)
id_list=pd.DataFrame(idlist)


In [73]:

def BookRecommender(movie_name = list(movies['title'].value_counts().index)):
    book_list_name = []
    book_id = movies[movies['title'] == movie_name].index
    book_id = book_id[0]
    for newid in idlist[book_id]:
        book_list_name.append(movies.loc[newid].title)
    return book_list_name



print(interact(BookRecommender))

interactive(children=(Dropdown(description='movie_name', options=('King Kong ', 'Cape Fear ', 'Parent Trap, Th…

<function BookRecommender at 0x00000184F96E39A0>
